In [1]:
import os
import pandas as pd
from qub_canvas_helper.calendar import Calendar

# Canvas Login Details

In [2]:
# Where on the internet is canvas
canvas_domain = "https://qub.instructure.com"

# Access token is now in .env under CANVAS_API_TOKEN
access_token = os.getenv("CANVAS_API_TOKEN")

# Check this is set up properly
if not access_token:
    raise ValueError("Missing Canvas API token!")

# Changes to Make    

Specify the course.

The course_id can be found in the url of the module: https://canvas.qub.ac.uk/courses/27862

In [3]:
course_id = 27862

Load the Excel file containing the custom calendar.

In [4]:
# Which excel file?
excel_calendar = "calendar_upload_test.xlsx"

# Specify the worksheet
sheet_name = "CHM1101"

The Excel file must be of a very specific layout, be sure of this beforehand!

In [5]:
# Load excel sheetm the first few rows do not contain timetable information
timetable_df = pd.read_excel(excel_calendar, sheet_name = sheet_name, skiprows = 4)

# The module code is usually in cell D2, extract this and add as a new column
module_code_df = pd.read_excel(excel_calendar, sheet_name=0, nrows=2, usecols='D')
module_code = module_code_df.iloc[0, 0]
timetable_df['Module Code'] = module_code

# CLean the data
timetable_df = timetable_df.dropna(subset=['Category'])

timetable_df.head()

,Week No.,Day,Date,Start Time,End Time,Topic,Notes,Staff,Room,Normal Capacity,Category,Module Code
0,8.0,Monday,2024-11-14 00:00:00,09:00:00,10:00:00,NaN,NaN,Notareal Person,DKB/OG/115,393,Lecture,CHM1101
1,9.0,Monday,2024-11-14 00:00:00,10:00:00,11:00:00,Tutorial,details will be in a separate e-mail,all Organic Lecturers,NaN,NaN,Tutorial,CHM1101
2,10.0,Monday,2024-11-21 00:00:00,10:00:00,11:00:00,Tutorial,details will be in a separate e-mail,all Organic Lecturers,NaN,NaN,Tutorial,CHM1101
3,12.0,Monday,2024-11-28 00:00:00,10:00:00,11:00:00,Tutorial,details will be in a separate e-mail,all Organic Lecturers,NaN,NaN,Tutorial,CHM1101
4,12.0,Tuesday,2024-11-16 00:00:00,11:00:00,13:00:00,NaN,NaN,Lecturer McLecturerFace,ASH/01/005,253,Workshop,CHM1101


Specify categories if required

In [6]:
# Categories to use
useful_categories = ("Workshop", "Lecture", "Labs")

# Filter the timetable by useful_categories
filtered_timetable_df = timetable_df[timetable_df['Category'].isin(useful_categories)]
filtered_timetable_df.head()

,Week No.,Day,Date,Start Time,End Time,Topic,Notes,Staff,Room,Normal Capacity,Category,Module Code
0,8.0,Monday,2024-11-14 00:00:00,09:00:00,10:00:00,NaN,NaN,Notareal Person,DKB/OG/115,393,Lecture,CHM1101
4,12.0,Tuesday,2024-11-16 00:00:00,11:00:00,13:00:00,NaN,NaN,Lecturer McLecturerFace,ASH/01/005,253,Workshop,CHM1101


# Upload a Custom Calendar

Instantiate the class using the data provided above.

In [7]:
calendar_manager = Calendar(canvas_domain, access_token, course_id)

See what assignments are already in the calendar. A date range must be specified, without the call defaults to today only.

In [8]:
calendar_manager.fetch_course_calendar_events(start_date="2024-10-14", end_date="2024-11-30")

,id,title,start_at,end_at,workflow_state,created_at,updated_at,all_day,all_day_date,comments,...,context_code,context_name,context_color,parent_event_id,hidden,child_events,url,html_url,duplicates,important_dates
0,131812,No Topic,2024-11-14T09:00:00Z,2024-11-14T10:00:00Z,active,2024-10-14T14:33:32Z,2024-10-14T14:33:32Z,False,None,None,...,course_27862,Paul Dingwall - Familiarisation Area,None,None,False,[],https://qub.instructure.com/api/v1/calendar_ev...,https://qub.instructure.com/calendar?event_id=...,[],False
1,131813,No Topic,2024-11-16T11:00:00Z,2024-11-16T13:00:00Z,active,2024-10-14T14:33:33Z,2024-10-14T14:33:33Z,False,None,None,...,course_27862,Paul Dingwall - Familiarisation Area,None,None,False,[],https://qub.instructure.com/api/v1/calendar_ev...,https://qub.instructure.com/calendar?event_id=...,[],False


Upload the timetable events outlined in the Excel file. If a conflict is detected, based on time slot, then no event is created.

In [9]:
calendar_manager.upload_calendar(filtered_timetable_df)

Conflict detected for No Topic on 2024-11-14 at 09:00. Skipping creation.
Conflict detected for No Topic on 2024-11-16 at 11:00. Skipping creation.


If mistakes are made, events can be removed. This will not remove assignments.

In [10]:
calendar_manager.remove_calendar_events(start_date="2024-10-14", end_date="2024-11-30")

Successfully removed event No Topic (ID: 131812) on 11/14/2024 at 09:00
Successfully removed event No Topic (ID: 131813) on 11/16/2024 at 11:00


Fetch event information by event id.

In [11]:
#calendar_manager.fetch_event_by_id(131366)